First, install the fhir.resources library.

In [3]:
!pip install fhir.resources

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 19.8 MB/s eta 0:00:00


The following script will extract observation resources from a questionnaire response resource.

In [35]:
import json
from decimal import Decimal
from fhir.resources.observation import Observation
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

def create_observation(link_id, text, answer):
    # Set code representing the question
    code = CodeableConcept(
        coding=[Coding(
            system="http://loinc.org",
            code=link_id.lstrip('/'),
            display=text
        )]
    )

    # Set status to 'final' as these are completed observations
    observation = Observation(code=code, status="final")

    # Set value based on the answer type
    if 'valueDecimal' in answer:
        value = float(answer['valueDecimal'])
        observation.valueQuantity = {
            "value": value,
            "unit": "kg" if "Weight" in text else "cm"
        }
    elif 'valueCodeableConcept' in answer:
        value_codeable_concept = CodeableConcept(
            coding=[Coding(
                system=answer['valueCodeableConcept']['system'],
                code=answer['valueCodeableConcept']['code'],
                display=answer['valueCodeableConcept']['display']
            )]
        )
        observation.valueCodeableConcept = value_codeable_concept

    return observation

def main():
    with open('qr_example.json', 'r') as f:
        data = json.load(f)

    observations = []

    # Iterate through each item
    for item in data['item']:
        link_id = item['linkId']
        text = item['text']
        answer = item['answer'][0]

        # Create Observation resource for the item
        observation = create_observation(link_id, text, answer)
        observations.append(observation)

    # Serialize Observation resources into JSON format
    for index, observation in enumerate(observations):
        # Convert Decimal values to floats before serialization
        observation_dict = observation.dict()
        for key, value in observation_dict.items():
            if isinstance(value, dict):
                for k, v in value.items():
                    if isinstance(v, Decimal):
                        value[k] = float(v)
            elif isinstance(value, Decimal):
                observation_dict[key] = float(value)

        with open(f'observation_{index + 1}.json', 'w') as f:
            json.dump(observation_dict, f, indent=4)

if __name__ == "__main__":
    main()
